In [1]:
from datasets import load_dataset, concatenate_datasets
from PIL import Image
import os
import numpy as np
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import io
import base64
import cv2
import json

In [2]:
ds = load_dataset("didi0di/finance-legal-mrc_merged-table")

README.md:   0%|          | 0.00/802 [00:00<?, ?B/s]

train-00000-of-00006.parquet:   0%|          | 0.00/36.1M [00:00<?, ?B/s]

train-00001-of-00006.parquet:   0%|          | 0.00/34.6M [00:00<?, ?B/s]

train-00002-of-00006.parquet:   0%|          | 0.00/32.6M [00:00<?, ?B/s]

train-00003-of-00006.parquet:   0%|          | 0.00/36.8M [00:00<?, ?B/s]

...

test-00000-of-00003.parquet:   0%|          | 0.00/36.1M [00:00<?, ?B/s]

test-00001-of-00003.parquet:   0%|          | 0.00/30.1M [00:00<?, ?B/s]

test-00002-of-00003.parquet:   0%|          | 0.00/42.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1457 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/743 [00:00<?, ? examples/s]

In [3]:
# train, validation, test, dev 모두 합치기 (없는것 무시)
def load_and_concatenate_datasets(dataset_):
    datasets = []
    for split in ['train', 'validation', 'test', 'dev']:
        if split in dataset_:
            ds_split = dataset_[split]
            datasets.append(ds_split)
    return concatenate_datasets(datasets)
ds = load_and_concatenate_datasets(ds)

In [8]:
ds.save_to_disk("finance-legal-mrc_merged-table")

Saving the dataset (0/8 shards):   0%|          | 0/2200 [00:00<?, ? examples/s]

In [7]:
ds[232]

{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=1024x305>,
 'table_title': '디지털 소비 현황 및 거래 유형별 현황',
 'table_html': '<table><tbody><tr><td rowspan="2">유형</td><td>2021년 조사(N=10,000)</td><td>2019년 조사(N=8,000)</td><td rowspan="2">2019년 대비 2021년 증감률</td></tr><tr><td colspan="2">디지털 소비자(%)</td></tr><tr><td>전체</td><td>82.1(8,207명)</td><td>44.0(3,523명)</td><td>+38.1%</td></tr><tr><td>TV홈쇼핑</td><td>40.3</td><td>27.3</td><td>+13.0%</td></tr><tr><td>인터넷·모바일 쇼핑</td><td>65.8</td><td>43.2</td><td>+22.6%</td></tr><tr><td>SNS 플랫폼 쇼핑</td><td>20.8</td><td>4.1</td><td>+16.7%</td></tr><tr><td>해외직구</td><td>9.4</td><td>6.5</td><td>+2.9%</td></tr><tr><td>개인간(C2C) 거래 플랫폼 쇼핑</td><td>16.6</td><td>-</td><td>-</td></tr></tbody></table>'}

In [ ]:
def display_sample(idx):
    data = ds[idx]
    img = Image.open(io.BytesIO(base64.b64decode(data["image"])))
    img_np = np.array(img)
    annots = data["annotations"]
    
    bboxes = annots['bboxes']        
    ner_tags = annots['ner_tags']
    words = annots['words']
    for bbox, ner_tag, word in zip(bboxes, ner_tags, words):
        if ner_tag != 0:  # Assuming 0 is the background class
            x1, y1, x2, y2 = bbox
            img_np = cv2.rectangle(img_np, (x1, y1), (x2, y2), (255, 0, 0), 2)
            img_np = cv2.putText(img_np, word, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
            
            
    display(Image.fromarray(img_np))
    

In [ ]:
slider = widgets.IntSlider(min=0, max=len(ds)-1, step=1, value=0)
widgets.interact(display_sample, idx=slider)

readingbank

In [2]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
ReadingBank Parquet 다운로드 전용 스크립트
"""

from pathlib import Path
from huggingface_hub import snapshot_download

# 1) NAS 목적지
TARGET_DIR = Path(
    "/mnt/AI_NAS/datalake/source/provider=huggingface/dataset=readingbank"
)
TARGET_DIR.mkdir(parents=True, exist_ok=True)

# 2) Parquet 파일만 snapshot
snapshot_path = snapshot_download(
    repo_id="zilongwang/ReadingBank",   # Parquet 버전이 올라가 있는 공개 repo
    repo_type="dataset",
    local_dir=str(TARGET_DIR),          # NAS 경로
    local_dir_use_symlinks=False,       # 실제 파일 복사
    allow_patterns=["*.parquet"],       # 🔑 Parquet 파일만 받기
    resume_download=True                # 중단돼도 이어받기
)

print(f"✅ 모든 Parquet 파일이 여기에 저장됐습니다 → {snapshot_path}")
print("💡 split-별 파일 예시:")
print("   train-00000-of-00005.parquet")
print("   dev-00000-of-00001.parquet")
print("   test-00000-of-00001.parquet")


/home/ian/anaconda3/envs/play/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/ian/anaconda3/envs/play/lib/python3.10/site-packages/huggingface_hub/file_download.py:980: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

dev-00000-of-00002.parquet:   0%|          | 0.00/91.8M [00:00<?, ?B/s]

train-00000-of-00015.parquet:   0%|          | 0.00/94.7M [00:00<?, ?B/s]

train-00001-of-00015.parquet:   0%|          | 0.00/93.5M [00:00<?, ?B/s]

...

train-00011-of-00015.parquet:   0%|          | 0.00/94.0M [00:00<?, ?B/s]

train-00012-of-00015.parquet:   0%|          | 0.00/94.7M [00:00<?, ?B/s]

train-00013-of-00015.parquet:   0%|          | 0.00/93.0M [00:00<?, ?B/s]

train-00014-of-00015.parquet:   0%|          | 0.00/91.1M [00:00<?, ?B/s]

✅ 모든 Parquet 파일이 여기에 저장됐습니다 → /mnt/AI_NAS/datalake/source/provider=huggingface/dataset=readingbank
💡 split-별 파일 예시:
   train-00000-of-00005.parquet
   dev-00000-of-00001.parquet
   test-00000-of-00001.parquet


In [3]:
from datasets import load_dataset

dataset = load_dataset(
    path="/mnt/AI_NAS/datalake/source/provider=huggingface/dataset=readingbank/readingbank.py",
    cache_dir="/mnt/AI_NAS/datalake/source/provider=huggingface/dataset=readingbank",
)

FileNotFoundError: https://layoutlm.blob.core.windows.net/readingbank/dataset/ReadingBank.zip